# TlF ground state spectrum

Using the Hamiltonian from Table 1 of [D.A. Wilkening, N.F. Ramsey, and D.J. Larson, Phys Rev A **29**, 425 (1984)](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.29.425).

![ramsey1](ramsey1.png)

In [1]:
import numpy as np
import sympy
from sympy import sqrt
import multiprocessing
import pickle

### Representing the states

A state, in general, can be written as a weighted superposition of the basis states. We work in the basis $|J, m_J, I_1, m_1, I_2, m_2\rangle$.

The operations we can define on the basis states are:

- construction: e.g. calling `BasisState(QN)` creates a basis state with quantum numbers `QN = (J, mJ, I1, m1, I2, m2)`;
- equality testing;
- inner product, returning either 0 or 1;
- superposition and scalar multiplication, returning a `State` object
- a convenience function to print out all quantum numbers

In [2]:
class BasisState:
    # constructor
    def __init__(self, J, mJ, I1, m1, I2, m2):
        self.J, self.mJ  = J, mJ
        self.I1, self.m1 = I1, m1
        self.I2, self.m2 = I2, m2

    # equality testing
    def __eq__(self, other):
        return self.J==other.J and self.mJ==other.mJ \
                    and self.I1==other.I1 and self.I2==other.I2 \
                    and self.m1==other.m1 and self.m2==other.m2

    # inner product
    def __matmul__(self, other):
        if self == other:
            return 1
        else:
            return 0

    # superposition: addition
    def __add__(self, other):
        if self == other:
            return State([ (2,self) ])
        else:
            return State([
                (1,self), (1,other)
            ])

    # superposition: subtraction
    def __sub__(self, other):
        return self + (-1)*other

    # scalar product (psi * a)
    def __mul__(self, a):
        return State([ (a, self) ])

    # scalar product (a * psi)
    def __rmul__(self, a):
        return self * a
    
    def print_quantum_numbers(self):
        print( self.J,"%+d"%self.mJ,"%+0.1f"%self.m1,"%+0.1f"%self.m2 )

A general state `State` can have any number of components, so let's represent it as an list of pairs `(amp, psi)`, where `amp` is the relative amplitude of a component, and `psi` is a basis state. The same component must not appear twice on the list.

There are three operations we can define on the states:

- construction
- superposition: concatenate component arrays and return a `State`
- scalar multiplication `a * psi` and `psi * a`, division, negation
- component-wise inner product `psi1 @ psi2`, where `psi1` is a bra, and `psi2` a ket, returning a complex number

In addition, I define an iterator method to loop through the components, and the `__getitem__()` method to access the components (which are not necessarily in any particular order!). See [Classes/Iterators](https://docs.python.org/3/tutorial/classes.html#iterators) for details.

In [3]:
class State:
    # constructor
    def __init__(self, data=[], remove_zero_amp_cpts=True):
        # check for duplicates
        for i in range(len(data)):
            amp1,cpt1 = data[i][0], data[i][1]
            for amp2,cpt2 in data[i+1:]:
                if cpt1 == cpt2:
                    raise AssertionError("duplicate components!")
        # remove components with zero amplitudes
        if remove_zero_amp_cpts:
            self.data = [(amp,cpt) for amp,cpt in data if amp!=0]
        else:
            self.data = data
        # for iteration over the State
        self.index = len(self.data)

    # superposition: addition
    # (highly inefficient and ugly but should work)
    def __add__(self, other):
        data = []
        # add components that are in self but not in other
        for amp1,cpt1 in self.data:
            only_in_self = True
            for amp2,cpt2 in other.data:
                if cpt2 == cpt1:
                    only_in_self = False
            if only_in_self:
                data.append((amp1,cpt1))
        # add components that are in other but not in self
        for amp1,cpt1 in other.data:
            only_in_other = True
            for amp2,cpt2 in self.data:
                if cpt2 == cpt1:
                    only_in_other = False
            if only_in_other:
                data.append((amp1,cpt1))
        # add components that are both in self and in other
        for amp1,cpt1 in self.data:
            for amp2,cpt2 in other.data:
                if cpt2 == cpt1:
                    data.append((amp1+amp2,cpt1))
        return State(data)
                
    # superposition: subtraction
    def __sub__(self, other):
        return self + -1*other

    # scalar product (psi * a)
    def __mul__(self, a):
        return State( [(a*amp,psi) for amp,psi in self.data] )

    # scalar product (a * psi)
    def __rmul__(self, a):
        return self * a
    
    # scalar division (psi / a)
    def __truediv__(self, a):
        return self * (1/a)
    
    # negation
    def __neg__(self):
        return -1 * self
    
    # inner product
    def __matmul__(self, other):
        result = 0
        for amp1,psi1 in self.data:
            for amp2,psi2 in other.data:
                result += amp1.conjugate()*amp2 * (psi1@psi2)
        return result

    # iterator methods
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.index == 0:
            raise StopIteration
        self.index -= 1
        return self.data[self.index]
    
    # direct access to a component
    def __getitem__(self, i):
        return self.data[i]

### Operators in Python

Define QM operators as Python functions that take `BasisState` objects, and return `State` objects. Since we are interested in finding matrix elements, we only need the action of operators on the basis states (but it'd be easy to generalize using a `for` loop).

The easiest operators to define are the diagonal ones $J^2, J_z, I_{1z}, I_{2z}$, which just multiply the state by their eigenvalue:

In [4]:
def J2(psi):
    return State([(psi.J*(psi.J+1),psi)])

def Jz(psi):
    return State([(psi.mJ,psi)])

def I1z(psi):
    return State([(psi.m1,psi)])

def I2z(psi):
    return State([(psi.m2,psi)])

The other angular momentum operators we can obtain through the ladder operators

$$ J_\pm=J_x\pm iJ_y. $$

These are defined through their action on the basis states as (Sakurai eqns 3.5.39-40)

$$ J_\pm|J,m\rangle=\sqrt{(j\mp m)(j\pm m+1)}|jm\pm1\rangle. $$

Similarly, $I_{1\pm},I_{2\pm}$ act on the $|I_1,m_1\rangle$ and $|I_2,m_2\rangle$ subspaces in the same way.

In [5]:
def Jp(psi):
    amp = sqrt((psi.J-psi.mJ)*(psi.J+psi.mJ+1))
    ket = BasisState(psi.J, psi.mJ+1, psi.I1, psi.m1, psi.I2, psi.m2)
    return State([(amp,ket)])

def Jm(psi):
    amp = sqrt((psi.J+psi.mJ)*(psi.J-psi.mJ+1))
    ket = BasisState(psi.J, psi.mJ-1, psi.I1, psi.m1, psi.I2, psi.m2)
    return State([(amp,ket)])

def I1p(psi):
    amp = sqrt((psi.I1-psi.m1)*(psi.I1+psi.m1+1))
    ket = BasisState(psi.J, psi.mJ, psi.I1, psi.m1+1, psi.I2, psi.m2)
    return State([(amp,ket)])

def I1m(psi):
    amp = sqrt((psi.I1+psi.m1)*(psi.I1-psi.m1+1))
    ket = BasisState(psi.J, psi.mJ, psi.I1, psi.m1-1, psi.I2, psi.m2)
    return State([(amp,ket)])

def I2p(psi):
    amp = sqrt((psi.I2-psi.m2)*(psi.I2+psi.m2+1))
    ket = BasisState(psi.J, psi.mJ, psi.I1, psi.m1, psi.I2, psi.m2+1)
    return State([(amp,ket)])

def I2m(psi):
    amp = sqrt((psi.I2+psi.m2)*(psi.I2-psi.m2+1))
    ket = BasisState(psi.J, psi.mJ, psi.I1, psi.m1, psi.I2, psi.m2-1)
    return State([(amp,ket)])

In terms of the above-defined ladder operators, we can write

$$J_x=\frac{1}{2}(J_++J_-);\quad
J_y=\frac{1}{2i}(J_+-J_-),$$

and similarly for $I_{1x}, I_{1y}$ and $I_{2x}, I_{2y}$.

In [6]:
def Jx(psi):
    return half*( Jp(psi) + Jm(psi) )

def Jy(psi):
    return -half*1j*( Jp(psi) - Jm(psi) )

def I1x(psi):
    return half*( I1p(psi) + I1m(psi) )

def I1y(psi):
    return -half*1j*( I1p(psi) - I1m(psi) )

def I2x(psi):
    return half*( I2p(psi) + I2m(psi) )

def I2y(psi):
    return -half*1j*( I2p(psi) - I2m(psi) )

### Composition of operators

All operators defined above can only accept `BasisStates` as their inputs, and they all return `States` as output. To allow composition of operators,

$$\hat A\hat B|\psi\rangle=\hat A(\hat B(|\psi\rangle)),$$

define the following function.

In [7]:
def com(A, B, psi):
    ABpsi = State()
    # operate with A on all components in B|psi>
    for amp,cpt in B(psi):
        ABpsi += amp * A(cpt)
    return ABpsi

### Rotational term

The simplest term in the Hamiltonian simply gives the rotational levels:

$$H_\text{rot}=B_\text{rot}\vec J^2.$$

In [8]:
def Hrot(psi):
    return Brot * J2(psi)

### Terms with ang. momentum dot products

Note that the dot product of two angular momentum operators can be written in terms of the ladder operators as

$$\vec A\cdot\vec B=A_zB_z+\frac{1}{2}(A_+B_-+A_-B_+).$$

We have the following terms (from Table 1 of Ramsey's paper):

$$
H_\text{c1}=c_1\vec I_1\cdot\vec J;\quad
H_\text{c2}=c_2\vec I_2\cdot\vec J;\quad
H_\text{c4}=c_4\vec I_1\cdot\vec I_2\\
H_\text{c3a}=15c_3\frac{(\vec I_1\cdot\vec J)(\vec I_2\cdot\vec J)}{(2J+3)(2J-1)}
=\frac{15c_3}{c_1c_2}\frac{H_\text{c1}H_\text{c2}}{(2J+3)(2J-1)}\\
H_\text{c3b}=15c_3\frac{(\vec I_2\cdot\vec J)(\vec I_1\cdot\vec J)}{(2J+3)(2J-1)}
=\frac{15c_3}{c_1c_2}\frac{H_\text{c2}H_\text{c1}}{(2J+3)(2J-1)}\\
H_\text{c3c}=-10c_3\frac{(\vec I_1\cdot\vec I_2)\vec J^2}{(2J+3)(2J-1)}
=\frac{-10c_3}{c_4 B_\text{rot}}\frac{H_\text{c4}H_\text{rot}}{(2J+3)(2J-1)}
$$

In [9]:
def Hc1(psi):
    return c1 * ( com(I1z,Jz,psi) + half*(com(I1p,Jm,psi)+com(I1m,Jp,psi)) )

def Hc2(psi):
    return c2 * ( com(I2z,Jz,psi) + half*(com(I2p,Jm,psi)+com(I2m,Jp,psi)) )

def Hc4(psi):
    return c4 * ( com(I1z,I2z,psi) + half*(com(I1p,I2m,psi)+com(I1m,I2p,psi)) )

def Hc3a(psi):
    return 15*c3/c1/c2 * com(Hc1,Hc2,psi) / ((2*psi.J+3)*(2*psi.J-1))

def Hc3b(psi):
    return 15*c3/c1/c2 * com(Hc2,Hc1,psi) / ((2*psi.J+3)*(2*psi.J-1))

def Hc3c(psi):
    return -10*c3/c4/Brot * com(Hc4,Hrot,psi) / ((2*psi.J+3)*(2*psi.J-1))

The overall field-free Hamiltonian is

In [10]:
def Hff(psi):
    return Hrot(psi) + Hc1(psi) + Hc2(psi) + Hc3a(psi) + Hc3b(psi) \
            + Hc3c(psi) + Hc4(psi)

### Zeeman Hamiltonian

In order to separate the task of finding the matrix elements and the eigenvalues, the Hamiltonian

$$H^\text{Z}=-\frac{\mu_J}{J}(\vec J\cdot\vec B)-\frac{\mu_1}{I_1}(\vec I_1\cdot\vec B)-\frac{\mu_2}{I_2}(\vec I_2\cdot\vec B)$$

is best split into three matrices:

$$H^\text{Z}=B_xH^\text{Z}_x+B_yH^\text{Z}_y+B_zH^\text{Z}_z,$$

where

$$ H^\text{Z}_x = -\frac{\mu_J}{J}J_x -\frac{\mu_1}{I_1}I_{1x} -\frac{\mu_2}{I_2}I_{2x} $$
$$ H^\text{Z}_y = -\frac{\mu_J}{J}J_y -\frac{\mu_1}{I_1}I_{1y} -\frac{\mu_2}{I_2}I_{2y} $$
$$ H^\text{Z}_z = -\frac{\mu_J}{J}J_z -\frac{\mu_1}{I_1}I_{1z} -\frac{\mu_2}{I_2}I_{2z} $$

Note that we are using the convention $\mu_1=\mu_\text{Tl}$ and $\mu_2=\mu_\text{F}$. The terms involving division by $J$ are only valid for states with $J\ne0$ (of course!).

In [11]:
def HZx(psi):
    if psi.J != 0:
        return -mu_J/psi.J*Jx(psi) - mu_Tl/psi.I1*I1x(psi) - mu_F/psi.I2*I2x(psi)
    else:
        return -mu_Tl/psi.I1*I1x(psi) - mu_F/psi.I2*I2x(psi)

def HZy(psi):
    if psi.J != 0:
        return -mu_J/psi.J*Jy(psi) - mu_Tl/psi.I1*I1y(psi) - mu_F/psi.I2*I2y(psi)
    else:
        return -mu_Tl/psi.I1*I1y(psi) - mu_F/psi.I2*I2y(psi)
    
def HZz(psi):
    if psi.J != 0:
        return -mu_J/psi.J*Jz(psi) - mu_Tl/psi.I1*I1z(psi) - mu_F/psi.I2*I2z(psi)
    else:
        return -mu_Tl/psi.I1*I1z(psi) - mu_F/psi.I2*I2z(psi)

### Stark Hamiltonian

Again splitting the Hamiltonian into the three spatial components, we have

$$H^\text{S}=-\vec d\cdot\vec E
= E_xH^\text{S}_x+E_yH^\text{S}_y+E_zH^\text{S}_z.
= e(E_{-1}r_{-1} + E_{0}r_{0} + E_{1}r_{1})
$$

The matrix elements for $r_p$ are given by

$$
\langle J', m_J', \Omega' |r_p| J, m_J, \Omega\rangle = \langle r_{q=0} \rangle (-1)^{m_J'} \mathrm{ThreeJ(J',1,J,-m_j',p,m_J)ThreeJ(J',1,J,0,0,0)} \left[(2J+1)(2J'+1)\right]^{\frac{1}{2}}
$$

and parity further enforces $\Delta J = \pm1$.

These three $r_p$ operators can be written in Python as the operators:

In [12]:
from sympy.physics.wigner import wigner_3j

def R10(psi):
    amp1 = ( (-1)**psi.mJ * wigner_3j(psi.J-1,1,psi.J,-psi.mJ,0,psi.mJ) * wigner_3j(psi.J-1,1,psi.J,0,0,0) 
            * sympy.sqrt(((2*psi.J+1)*(2*(psi.J-1)+1))) )
    ket1 = BasisState(psi.J-1, psi.mJ, psi.I1, psi.m1, psi.I2, psi.m2)
    amp2 = ( (-1)**psi.mJ * wigner_3j(psi.J+1,1,psi.J,-psi.mJ,0,psi.mJ) * wigner_3j(psi.J+1,1,psi.J,0,0,0) 
            * sympy.sqrt(((2*psi.J+1)*(2*(psi.J+1)+1))) )
    ket2 = BasisState(psi.J+1, psi.mJ, psi.I1, psi.m1, psi.I2, psi.m2)
    return State([(amp1,ket1),(amp2,ket2)])

def R1m(psi):
    amp1 = ( (-1)**(psi.mJ-1) * wigner_3j(psi.J-1,1,psi.J,-(psi.mJ-1),-1,psi.mJ) * wigner_3j(psi.J-1,1,psi.J,0,0,0) 
            * sympy.sqrt(((2*psi.J+1)*(2*(psi.J-1)+1))) )
    ket1 = BasisState(psi.J-1, psi.mJ-1, psi.I1, psi.m1, psi.I2, psi.m2)
    amp2 = ( (-1)**(psi.mJ-1) * wigner_3j(psi.J+1,1,psi.J,-(psi.mJ-1),-1,psi.mJ) * wigner_3j(psi.J+1,1,psi.J,0,0,0) 
            * sympy.sqrt(((2*psi.J+1)*(2*(psi.J+1)+1))) )
    ket2 = BasisState(psi.J+1, psi.mJ-1, psi.I1, psi.m1, psi.I2, psi.m2)
    return State([(amp1,ket1),(amp2,ket2)])

def R1p(psi):
    amp1 = ( (-1)**(psi.mJ+1) * wigner_3j(psi.J-1,1,psi.J,-(psi.mJ+1),1,psi.mJ) * wigner_3j(psi.J-1,1,psi.J,0,0,0) 
            * sympy.sqrt(((2*psi.J+1)*(2*(psi.J-1)+1))) )
    ket1 = BasisState(psi.J-1, psi.mJ+1, psi.I1, psi.m1, psi.I2, psi.m2)
    amp2 = ( (-1)**(psi.mJ+1) * wigner_3j(psi.J+1,1,psi.J,-(psi.mJ+1),1,psi.mJ) * wigner_3j(psi.J+1,1,psi.J,0,0,0) 
            * sympy.sqrt(((2*psi.J+1)*(2*(psi.J+1)+1))) )
    ket2 = BasisState(psi.J+1, psi.mJ+1, psi.I1, psi.m1, psi.I2, psi.m2)
    return State([(amp1,ket1),(amp2,ket2)])

In terms of the operators

$$
R_{\pm}\equiv \mp\frac{x\pm iy}{\sqrt2r} = 2\sqrt{\frac{\pi}{3}}Y_1^{\pm M} \\
R_{0}\equiv \frac{z}{r} = 2\sqrt{\frac{\pi}{3}}Y_1^{0}
$$
and the molecular dipole moment $d_\text{TlF}$, the three Stark Hamiltonians are

$$
\begin{align}
H^\text{S}_x&=-d_\text{TlF}(R^{-1}_1-R^1_1)/\sqrt2\\
H^\text{S}_y&=-d_\text{TlF}i(+R^{-1}_1+R^1_1)/\sqrt2\\
H^\text{S}_z&=-d_\text{TlF}R^0_1
\end{align}
$$

In Python:

In [13]:
def HSx(psi):
    return -D_TlF * ( R1m(psi) - R1p(psi) ) / np.sqrt(2)

def HSy(psi):
    return -D_TlF * 1j * ( R1m(psi) + R1p(psi) ) / np.sqrt(2)

def HSz(psi):
    return -D_TlF *R10(psi)
                    

### An alternative $c_3$ term

The $c_3$ term in Ramsey's Hamiltonian assumes that $J$ is a good quantum number, which breaks down at high $E$ field. From [Wikipedia](https://en.wikipedia.org/wiki/Hyperfine_structure#Molecular_hyperfine_structure), we get the term

$$
H_\text{c3}^\text{alt}=\frac{5}{2}c_3\left[
2\vec I_1\vec I_2
-3(\vec I_1\cdot\vec{\hat{R}})(\vec I_2\cdot\vec{\hat{R}})
-3(\vec I_2\cdot\vec{\hat{R}})(\vec I_1\cdot\vec{\hat{R}})
\right].
$$

Write the dot products in the form

$$
H_\text{I1R}=\vec I_1\cdot\vec{\hat{R}}=I_{1z}R_1^0+\frac{1}{\sqrt{2}}(I_{1+}R_1^{-1}-I_{1-}R_1^{1})\\
H_\text{I2R}=\vec I_2\cdot\vec{\hat{R}}=I_{2z}R_1^0+\frac{1}{\sqrt{2}}(I_{2+}R_1^{-1}-I_{2-}R_1^{1}),
$$

as follows:

In [14]:
def HI1R(psi):
    return com(I1z,R10,psi) + sqrt(2)*half*(com(I1p,R1m,psi)-com(I1m,R1p,psi))

def HI2R(psi):
    return com(I2z,R10,psi) + sqrt(2)*half*(com(I2p,R1m,psi)-com(I2m,R1p,psi))

The $c_3$ term becomes

$$
H_\text{c3}^\text{alt}=\frac{5}{2}c_3\left[
\frac{2}{c_4}H_\text{c4}-3H_\text{I1R}H_\text{I2R}-3H_\text{I2R}H_\text{I1R}
\right].
$$

In Python:

In [15]:
def Hc3_alt(psi):
    return 5*c3/c4*Hc4(psi) - 15*c3/2*(com(HI1R,HI2R,psi)+com(HI2R,HI1R,psi))

The corresponding alternative field-free Hamiltonian is

In [16]:
def Hff_alt(psi):
    return Hrot(psi) + Hc1(psi) + Hc2(psi) + Hc3_alt(psi) + Hc4(psi)

### Finding the matrix elements

With all the operators defined, we can evaluate the matrix elements for a given range of quantum numbers. We shall need to generate a non-integer range list (e.g., from -3/2 to +3/2):

In [17]:
def ni_range(x0, x1, dx=1):
    # sanity check arguments
    if dx==0:
        raise ValueError("invalid parameters: dx==0")
    if x0>x1 and dx>=0:
        raise ValueError("invalid parameters: x0>x1 and dx>=0")
    if x0<x1 and dx<=0:
        raise ValueError("invalid parameters: x0<x1 and dx<=0")
        
    # generate range list
    range_list = []
    x = x0
    while x < x1:
        range_list.append(sympy.Number(x))
        x += dx
    return range_list

Define constants for TlF:

In [18]:
half = sympy.Rational(1,2)
#half = 0.5

Jmin = sympy.Integer(0)
Jmax = sympy.Integer(10) # max J value in Hamiltonian
#Jmax = 6
I_Tl = half             # I1 in Ramsey's notation
I_F  = half             # I2 in Ramsey's notation

Brot = sympy.symbols('Brot')
c1, c2, c3, c4 = sympy.symbols('c1 c2 c3 c4')
D_TlF = sympy.symbols('D_TlF')
mu_J, mu_Tl, mu_F = sympy.symbols('mu_J mu_Tl mu_F')

 Write down the basis as a list of `BasisState` components:

In [20]:
# QN = [BasisState(J,mJ,I_Tl,m1,I_F,m2)
#       for J  in ni_range(0, Jmax+1)
#       for mJ in ni_range(-J,J+1)
#       for m1 in ni_range(-I_Tl,I_Tl+1)
#       for m2 in ni_range(-I_F,I_F+1)
#      ]

QN = []

for J in np.arange(Jmin, Jmax+1):
    for m1 in np.arange(-I_Tl,I_Tl+1):
        for m2 in np.arange(-I_F,I_F+1):
            mJ = 0-m1-m2
            
            if np.abs(mJ) <= J:
                QN.append(BasisState(J,mJ,I_Tl,m1,I_F,m2))

The field-free and Stark/Zeeman components of the Hamiltonian then have the matrix elements (evaluate using `multiprocessing` to speed things up)

In [21]:
%%time
from tqdm import tqdm

def HMatElems(H, QN=QN):
    result = sympy.zeros(len(QN),len(QN))
    for i,a in tqdm(enumerate(QN)):
        for j,b in enumerate(QN):
            result[i,j] = (1*a)@H(b)
    return result

H_ops = [Hff, HSx, HSy, HSz, HZx, HZy, HZz]
Hff_m, HSx_m, HSy_m, HSz_m, HZx_m, HZy_m, HZz_m = map(HMatElems, H_ops)

42it [00:03, 13.55it/s]
42it [00:02, 20.73it/s]
42it [00:01, 21.31it/s]
42it [00:00, 47.54it/s]
42it [00:00, 86.81it/s]
42it [00:00, 53.76it/s]
42it [00:00, 170.59it/s]

Wall time: 9.54 s


Store the result of the calculation as text files and Python `pickle`s:

In [22]:
with open("./Saved Hamiltonians/TlF_X_state_hamiltonian_J0to10_mF0.pickle", 'wb') as f:
    pickle.dump(
        {
            "Hff" : Hff_m,
            "HSx" : HSx_m,
            "HSy" : HSy_m,
            "HSz" : HSz_m,
            "HZx" : HZx_m,
            "HZy" : HZy_m,
            "HZz" : HZz_m,
        },
        f
    )

## Quantum operators
To speed up calculations of e.g. spin expectation values, it is useful to have the spin operators as matrices. Calculating them here.

In [23]:
def MatElems(A, QN=QN):
    result = np.zeros((len(QN),len(QN)),dtype = complex)
    for i,a in tqdm(enumerate(QN)):
        for j,b in enumerate(QN):
            result[i,j] = (1*a)@A(b)
    return result

spin_ops = [Jx,Jy,Jz,I1x,I1y,I1z,I2x,I2y,I2z]
Jx_m,Jy_m,Jz_m,I1x_m,I1y_m,I1z_m,I2x_m,I2y_m,I2z_m = map(MatElems, spin_ops)

196it [00:03, 62.28it/s]
196it [00:05, 34.94it/s]
196it [00:00, 263.20it/s]
196it [00:02, 79.18it/s]
196it [00:04, 40.53it/s]
196it [00:01, 181.32it/s]
196it [00:02, 81.42it/s]
196it [00:04, 40.64it/s]
196it [00:01, 183.53it/s]


In [24]:
with open("spin_operators.py", 'wb') as f:
    pickle.dump(
        {
            "Jx" : Jx_m,
            "Jy" : Jy_m,
            "Jz" : Jz_m,
            "I1x" : I1x_m,
            "I1y" : I1y_m,
            "I1z" : I1z_m,
            "I2x" : I2x_m,
            "I2y" : I2y_m,
            "I2z" : I2z_m,
        },
        f
    )

In [25]:
Jx_m

array([[0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       ...,
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j]])